qgrid - SlickGrid in Jupyter Notebooks
======================================
Qgrid is an Jupyter notebook widget which uses a javascript library called SlickGrid to render pandas DataFrames within a Jupyter notebook.  It was developed for use in [Quantopian's hosted research environment]( https://www.quantopian.com/research?utm_source=github&utm_medium=web&utm_campaign=qgrid-nbviewer).

The purpose of this notebook is to give an overview of what qgrid is capable of.  In case you're wondering why every cell has redundant `import` statementsj it's convenient while testing to not have to run an import cell at the top of the notebook every time the kernel is restarted.

## Overview
* [SlickGrid](https://github.com/mleibman/SlickGrid) is a javascript grid which allows users to scroll, sort, 
and filter hundreds of thousands of rows with extreme responsiveness.  
* [Pandas](https://github.com/pydata/pandas) is a powerful data analysis / manipulation library for Python, and DataFrames are the primary way of storing and manipulating two-dimensional data in pandas.

[Qgrid](https://github.com/quantopian/qgrid) renders pandas DataFrames as SlickGrids, which enables users to explore the entire contents of a DataFrame using intuitive sorting and filtering controls.  It's designed to be used with Jupyter notebook, Jupyterhub, or Jupyterlab, and is also fully fuctional on [mybinder.org](http://mybinder.org/).

## API & Usage
API documentation is hosted on [readthedocs](http://qgrid.readthedocs.io/en/widget-guidelines/). 

The API documentation can also be accessed via the "?" operator in IPython.  To use the "?" operator, type the name of the function followed by "?" to see the documentation for that function, like this:
```
qgrid.show_grid?
qgrid.set_defaults?
qgrid.set_grid_options?
qgrid.enable?
qgrid.disable?

```

## Example 1 - DataFrame with many different types of columns
Create a sample DataFrame and render it using qgrid:

In [8]:
import numpy as np
import pandas as pd
import qgrid
%config Application.log_level="INFO"
randn = np.random.randn
df3 = pd.DataFrame({
    'A' : 1.,
    'B' : pd.Series(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08', '2013-01-09'],index=list(range(9)),dtype='datetime64[ns]'),
    'C' : pd.Series(randn(9),index=list(range(9)),dtype='float32'),
    'D' : np.array([3] * 9,dtype='int32'),
    'E' : pd.Categorical(["washington", "adams", "washington", "madison", "lincoln","jefferson", "hamilton", "roosevelt", "kennedy"]),
    'F' : ["foo", "bar", "buzz", "bippity","boppity", "foo", "foo", "bar", "zoo"] })
widget = qgrid.show_grid(df3, show_toolbar=True)
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8}, precision=5, show_toolbar=True)

If you make any sorting/filtering changes, or edit the grid by double clicking, you can retrieve a copy of your DataFrame which reflects these changes by calling `get_changed_df` on the `QgridWidget` instance returned by `show_grid`.

In [9]:
widget.get_changed_df()

,A,B,C,D,E,F
0,1.0,2013-01-01,-0.471098,3,washington,foo
1,1.0,2013-01-02,-0.816563,3,adams,bar
2,1.0,2013-01-03,0.852915,3,washington,buzz
3,1.0,2013-01-04,-0.555588,3,madison,bippity
...,...,...,...,...,...,...
5,1.0,2013-01-06,0.581864,3,jefferson,foo
6,1.0,2013-01-07,0.501413,3,hamilton,foo
7,1.0,2013-01-08,-1.301142,3,roosevelt,bar
8,1.0,2013-01-09,0.112007,3,kennedy,zoo


## Example 2 - Render a DataFrame with 1 million rows

Create the DataFrame and render it using qgrid:

In [10]:
import pandas as pd
import numpy as np
import qgrid

df_rand = pd.DataFrame(np.random.randn(1000000, 4), columns=list('ABCD'))
# duplicate column B as a string column, to test scalability for text column filters
df_rand['B (as str)'] = df_rand['B'].map(lambda x: str(x))
q = qgrid.show_grid(df_rand, show_toolbar=True, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200, 'maxVisibleRows': 10})
q

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defaultColumnWidth': 200, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 10, 'minVisibleRows': 8}, precision=5, show_toolbar=True)

## Example 3 - Render a DataFrame returned by Yahoo Finance

Create a sample DataFrame using the `get_data_yahoo` function and render it without using qgrid

In [14]:
import pandas as pd
import numpy as np
import qgrid
randn = np.random.randn

# Set this pandas option to prevent the grid from being too big
pd.set_option('display.max_rows', 8)

# Get a pandas DataFrame containing the daily prices for the S&P 500 from 1/1/2011 - 1/1/2014
from pandas_datareader.data import DataReader
spy = DataReader(
    'SPY',
    'yahoo',
    pd.Timestamp('2011-01-01'),  
    pd.Timestamp('2014-01-01'),
)
widget = qgrid.show_grid(spy, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defaultColumnWidth': 200, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8}, precision=5)

## Example 4 - Render a DataFrame with a multi-index

Create a sample DataFrame using the `wb.download` function and render it without using qgrid

In [15]:
import qgrid
import pandas as pd
from pandas_datareader import wb
df2 = wb.download(indicator='NY.GDP.PCAP.KD', country=['all'], start=2005, end=2008)
df2.columns = ['GDP per capita (constant 2005 US$)']
qgrid.show_grid(df2)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8}, precision=5)

## Example 6 - Render a DataFrame with an interval column
Create a sample DataFrame using the `wb.download` function and render it without using qgrid

In [16]:
import numpy as np
import pandas as pd
import qgrid

td = np.cumsum(np.random.randint(1, 15*60, 1000))
start = pd.Timestamp('2017-04-17')
df = pd.DataFrame(
    [(start + pd.Timedelta(seconds=d)) for d in td],
    columns=['time'])

freq = '15Min'
start = df['time'].min().floor(freq)
end = df['time'].max().ceil(freq)
bins = pd.date_range(start, end, freq=freq)

df['time_bin'] = pd.cut(df['time'], bins)

qgrid.show_grid(df, show_toolbar=True)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8}, precision=5, show_toolbar=True)

## Example 7 - Render a sample DataFrame with a multi-index
Create a sample DataFrame using the `wb.download` function and render it without using qgrid

In [17]:
import numpy as np
import pandas as pd
import qgrid

arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
df_multi = pd.DataFrame(np.random.randn(8, 4), index=arrays)
qgrid.show_grid(df_multi)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8}, precision=5)